In [191]:
import json

In [110]:
df = pd.read_csv("news.csv")

In [154]:
df = df[["title", "description","text"]]

In [159]:
data_dicts = []
for i in range(len(df)):
    data = df.iloc[i,:]
    data = {"title": data["title"], "description" : data["description"], "text" : data["text"]}
    data_dicts.append(data)

In [192]:
from elasticsearch import Elasticsearch

In [193]:
INDEX_NAME = "covidnews"
ES_HOST = {"host" : "localhost", "port" : 9200}

In [194]:
es = Elasticsearch(hosts= [ES_HOST])

In [195]:
if es.indices.exists(INDEX_NAME):
    print("deleting '%s' index..." % (INDEX_NAME))
    res = es.indices.delete(index = INDEX_NAME)
    print(" response: '%s'" % (res))

deleting 'covidnews' index...
 response: '{'acknowledged': True}'


In [196]:
request_body = {
    "settings" : {
        "number_of_shards": 1,
        "number_of_replicas": 0
    }
}
print("creating '%s' index..." % (INDEX_NAME))
res = es.indices.create(index = INDEX_NAME, body = request_body)
print(" response: '%s'" % (res))

creating 'covidnews' index...
 response: '{'acknowledged': True, 'shards_acknowledged': True, 'index': 'covidnews'}'


In [197]:
from tqdm import tqdm

In [ ]:
ids = 1
errors = 0
for data in tqdm(data_dicts):
    try:
        res = es.index(index='covidnews',doc_type='articles',id=ids,body=data)
    except:
        errors += 1
        print(data)
    ids += 1

  0%|                                                                                         | 0/6788 [00:00<?, ?it/s]C:\Users\rjkin\AppData\Local\Programs\Python\Python36\lib\site-packages\elasticsearch\connection\base.py:190: ElasticsearchDeprecationWarning: [types removal] Specifying types in document index requests is deprecated, use the typeless endpoints instead (/{index}/_doc/{id}, /{index}/_doc, or /{index}/_create/{id}).
  warnings.warn(message, category=ElasticsearchDeprecationWarning)
 38%|█████████████████████████████▋                                                | 2589/6788 [04:55<07:33,  9.26it/s]

{'title': 'NYC hospitals could run out of COVID-19 test swabs, city health department memo says', 'description': 'New York City is in danger of running out of swabs for COVID-19 tests and is urging health-care providers to continue testing only patients who are gravely ill, the city health department said in a memo.', 'text': 'New York City is in danger of running out of swabs for COVID-19 tests and is urging health-care\xa0providers to continue testing only patients who are gravely ill, the city health department said in a memo. "As the swab supply continues to decline, there is a real possibility hospitals will completely run out," the April 11\xa0alert to health-care providers said. "At this time, providers are reminded to only test hospitalized patients in order to preserve resources that are needed to diagnose and appropriately manage patients with more severe illness." The warning came amid repeated pleas from New York City and state officials for the federal government to provid

 39%|██████████████████████████████▌                                               | 2657/6788 [05:03<07:13,  9.52it/s]

{'title': 'Get informed on the top stories of the day in one quick scan', 'description': "In today's Morning Brief, we look at the debate over how and when to ease physical distancing rules. We also have stories on the drop in oversight of infection control measures at long-term care facilities in Ontario, and why China suddenly reported a 50 per cent in the number of deaths in Wuhan, the epicentre of the coronavirus outbreak.", 'text': 'Good morning! This is our daily news roundup\xa0with everything you need to know in one concise read. Sign up here to get this delivered to your inbox every morning. Will it really take \'weeks\' to ease physical distancing? Maybe not With increasing evidence that Canada is flattening its\xa0COVID-19 curve, the next challenge is when and how to begin easing the country\'s physical distancing rules. Take it slow and steady, say\xa0some experts, who note\xa0that it might be possible to start the easing sooner than you think.  There might well have been c

 39%|██████████████████████████████▌                                               | 2663/6788 [05:03<08:18,  8.27it/s]

{'title': 'What you need to know about COVID-19 in Ottawa on Thursday, March 26', 'description': "CBC Ottawa's latest roundup of key points during the coronavirus pandemic.", 'text': 'Recent developments:  Via Rial employee who worked on Quebec City-Windsor route tests positive for COVID-19 University of Ottawa cancels spring convocation.\xa0 2 more deaths, 170 new cases of COVID-19 in Ontario. 8 new confirmed cases of COVID-19 in the region; 2 are in hospital. Ottawa, Gatineau closing community sports parks and courts, playgrounds. Gatineau opening more shelter space to encourage physical distancing. Ontario restaurants can now offer takeout or delivery alcohol. Montfort and\xa0Queensway Carleton hospitals prepare to open centres for\xa0assessment and clinical care of\xa0COVID-19 patients.\xa0 The Ottawa Hospital is appealing for donations of masks, gloves, shields and other personal protective\xa0equipment for health-care staff. Any companies or organizations with donations can email

 39%|██████████████████████████████▋                                               | 2673/6788 [05:04<06:44, 10.17it/s]

{'title': 'Montreal firefighters to work 24-hour shifts, give up vacations in response to COVID-19 crisis', 'description': 'With five Montreal firefighters infected and 35 others in isolation, firefighters will respond to only the most critical 911 emergencies for now, as part of a host of measures to prevent further\xa0spread of COVID-19 within that vital workforce.', 'text': 'Montreal firefighters will respond respond to only the most critical 911 emergencies for the duration of the COVID-19 outbreak, as part of a host of measures to prevent further\xa0spread of COVID-19 within that vital workforce. As of Friday morning, five Montreal firefighters had tested positive for the novel coronavirus. They\xa0worked at stations in Westmount, Ahuntsic and Parc-Extension. They are now in self-isolation, along with 35 other firefighters who were in direct contact with them. "The isolation of certain firefighters was also necessary after a cardiac emergency intervention for a citizen that, in al

 40%|███████████████████████████████▍                                              | 2738/6788 [05:11<08:33,  7.89it/s]

In [125]:
search = {"size": 4,"query": {"match": {"text": "china death in covid-19"}}}
res= es.search(index= INDEX_NAME,body = json.dumps(search))

In [126]:
res["hits"]

{'total': {'value': 6784, 'relation': 'eq'},
 'max_score': 6.495981,
 'hits': [{'_index': 'covidnews',
   '_type': 'articles',
   '_id': '526',
   '_score': 6.495981,
   '_source': {'title': 'Timeline: How the new coronavirus arose and spread',
    'description': "Chinese authorities first investigated an outbreak of respiratory illness in the central city of Wuhan in late December. Here's a look at how information about its cause has unfolded and how it has spread since then.",
    'text': 'Chinese authorities first investigated an outbreak of respiratory illness in the central city of Wuhan in late December. Here\'s a look at how information about its cause has unfolded and how it has spread since then.   Dec. 31, 2019  Chinese experts investigate an outbreak of respiratory illness in the central city of Wuhan. Most patients had visited a seafood market in the city. Authoritiesshutit down and disinfected it. Total reported cases: 27 Deaths: 0  Jan. 3, 2020  Chinese officials rule out

In [190]:
for hit in res["hits"]["hits"]:
    print("id: {}, score: {}".format(hit["_id"], hit["_score"]))
    print(hit["_source"]["title"])
    print()

id: 4467, score: 15.379805
No, you can't make an N95 mask out of a bra

id: 5994, score: 11.361135
Manitoba forecast eases fears of major flooding during COVID-19 pandemic

id: 900, score: 9.870743
No running water on Air Canada flight from China during worsening coronavirus outbreak

id: 5220, score: 9.804422
300 Cache Creek residents on evacuation alert due to fear of flooding



In [37]:
T = df.loc[12,"text"]

In [35]:
from spacy.lang.en import English
nlp = English()
nlp.add_pipe(nlp.create_pipe('sentencizer'))
MAX_LEN = 512

In [97]:
def solve_window(model, claim : str, text : str):
    pass


def make_windows(text : str):
    doc = nlp(text)
    sentences = [sent.string.strip() for sent in doc.sents]
    total_sents = len(sentences)

    # Checking if any sentence has a length > MAX_LENGTH if it has then truncate the sentence 


    truncate = lambda sent : sent[:MAX_LEN]
    sentences = list(map(truncate,sentences))

    
    sent_lengths = [len(s.split()) for s in sentences]
    
    chunks = []

    window_start = 0
    window_end = 0
    word_count = 0
    while window_end < total_sents:

        if(sent_lengths[window_end] + word_count > 512):
            temp_str = ""
            for i in range(window_start, window_end):
                temp_str += sentences[i]
            chunks.append(temp_str)
            
            window_start += 1
            word_count -= sent_lengths[window_start]
        else:
            word_count += sent_lengths[window_end]
            window_end += 1
    
    
    
    temp_str = ""
    for i in range(window_start, window_end):
        temp_str += sentences[i]
        
    chunks.append(temp_str)
    return chunks
                



In [102]:
from tqdm import tqdm

In [107]:
sm= 0
for T in tqdm(df.loc[:,"text"]):
    W = make_windows(T)
    sm += len(W)

100%|█████████████████████████████████████████████████████████████████████████████| 6788/6788 [00:42<00:00, 159.36it/s]


In [108]:
sm/len(df)

14.831761932822628

In [111]:
sm = 0
for sent in df.loc[:,"text"]:
    if(len(sent.split()) > 512):
        sm += 1

In [112]:
sm

4554

In [146]:
print(df.loc[398,"text"])

Five British nationals, including a nine-year-old child, have been diagnosed with the coronavirus in France, after staying in the same ski chalet and coming into contact with a person who had been in Singapore, Health Minister Agnès Buzyn said on Saturday. The total number of people infected with the virus in France has now reached 11. Buzyn said the group of people newly infected with the virus were not in a serious condition. They had formed "a cluster, a grouping around one original case" after staying in the same chalet, in the Contamines-Montjoie resort in Savoie in eastern France. "That original case was brought to our attention last night, it is a British national who had returned from Singapore where he had stayed between January 20 and 23, and he arrived in France on January 24 for four days," Buzyn said. As China deaths hit 722, WHO warns of global shortage of coronavirus protective equipmentWhat we know so far about the coronavirus Two of the apartments in the ski chalet wer

In [145]:
df.loc[123,"text"].replace("\xa0","")

'The latest:  Worldwide COVID-19 death toll tops 100,000. U.S. sees more than 35,000 new cases and 2,108 deaths on Friday. Government documents reveal a slow start to Canada\'s COVID-19 response. Canadian military intelligence unit issued warning about Wuhan outbreak back in January. Emergencies Act talks rejected by premiers, deemed a distraction. Health experts are calling for increased COVID-19 testing—so why isn\'t Canada testing everyone? INTERACTIVE |Tracking the coronavirus in Canada.  The worldwide death toll from the coronavirus topped 100,000 on Friday, with the U.S. seeing more than 2,000 deaths in one day,according to the running tally kept by Johns Hopkins University. The sad milestone comes as Christians around the globe mark a Good Friday unlike any other —in front of computer screens instead of in church pews. Meanwhile, some countries are tiptoeing toward reopening segments of their battered economies. In Canada, a physician who tweeted about an elderly patient who cho

In [1]:
import requests
from bs4 import BeautifulSoup
import re
import newspaper
from collections import deque
Article = newspaper.Article
import sys
from tqdm import tqdm
import pickle
import os


In [2]:
def save_dict(obj, path ):
    with open(path, 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_dict(path):
    with open(path, 'rb') as f:
        return pickle.load(f)




In [48]:
class LinkExtractor:
    def __init__(self, name : str, base_url : str, domain_name : str = None,
                 max_links : int = 100000, avoid_patterns : list = None , accept_if_only_patterns : list = None):
        self.name = name
        self.base_url = base_url
        self.max_links = max_links
        self.avoid_patterns = avoid_patterns
        self.accept_if_only_patterns = accept_if_only_patterns
        if(domain_name == None):
            self.domain_name = base_url.split("://")[1]
        self.all_urls = set()
        
        if(avoid_patterns is not None):
            for patt in avoid_patterns:
                if(type(patt) != type(re.compile("type_check"))):
                    assert False , "avoid_list only takes compiled re objects"
        if(accept_if_only_patterns is not None):
            for patt in accept_if_only_patterns:
                if(type(patt) != type(re.compile("type_check"))):
                    assert False , "avoid_list only takes compiled re objects"
        
        
    def __str__(self):
        return self.name


    
    def _does_match_avoid(self,string : str):
        if(self.avoid_patterns is None):
            return False
        for patt in self.avoid_patterns:
            if(not patt.fullmatch(string) is None):
                return True
        
        return False
            
    
    def _does_match_accept(self,string : str):
        if(self.accept_if_only_patterns is None):
            return True
        for patt in self.accept_if_only_patterns:
            if(patt.fullmatch(string) is not None):
                return True
        
        return False
    
    def extract_links(self, url: str):
        try:
            response = requests.get(url, timeout = 1)
        except:
            return []
        if(response.status_code != 200):
            return []
        soup = BeautifulSoup(response.text)
        links = []
        for link in soup.findAll('a', attrs={'href': re.compile("^http://|^https://")}):
            l = link.get("href")
            prefixes = ["https://" + self.domain_name , "http://" + self.domain_name]
            if(l.startswith(prefixes[0]) or l.startswith(prefixes[1])):
                links.append(link.get('href'))
        return links
    
    def extract_all_links(self):
        # doing a bfs here
        
        
        seen_urls = set()
        seen_urls.add(self.base_url)
        queue = deque()
        queue.appendleft(self.base_url)
        
        self.all_urls.add(self.base_url)
        
        while len(queue) > 0 and len(self.all_urls) < self.max_links:
            sys.stdout.write('\r'+ "total visited = {} found = {}".format(len(seen_urls), len(self.all_urls)))
           # print(len(self.all_urls))
            current_url = queue.pop()
            urls_local = self.extract_links(current_url)
            
            for url in urls_local:
                if(url not in seen_urls):
                    queue.appendleft(url)
                    seen_urls.add(url)
                    if(self._does_match_accept(url) and not self._does_match_avoid(url)):
                        self.all_urls.add(url)
        
        
    
    
    

In [49]:
"hello world".startswith("hello")

True

In [50]:
NDTV_extractor = LinkExtractor(name = "NDTV", base_url = "https://www.ndtv.com/", avoid_patterns = [re.compile(r".*/page-[0-9]+"), re.compile(r".*/video.*")],
                accept_if_only_patterns = [ re.compile(r".*-[0-9]*")]
             )

In [66]:
ITV_extractor = LinkExtractor(name = "TOI", base_url = "https://www.indiatvnews.com/",
                              avoid_patterns = [re.compile(r".*(/photos|/video|/brand-content).*")],
                              accept_if_only_patterns = [re.compile(r".*-[0-9]*")])

In [67]:
ITV_extractor.extract_all_links()

total visited = 33115 found = 16154

KeyboardInterrupt: 

In [68]:
with open("ITV_urls.txt", "w") as f:
    for url in ITV_extractor.all_urls:
        f.write(url + "\n")

In [65]:
ITV_extractor.all_urls

{'https://www.indiatvnews.com/',
 'https://www.indiatvnews.com/auto/all-new-honda-city-2020-launched-check-ex-showroom-price-and-details-646416',
 'https://www.indiatvnews.com/auto/audi-q2-bookings-open-in-india-q2-suv-price-features-details-654043',
 'https://www.indiatvnews.com/auto/audi-rs7-launched-in-india-priced-at-rs-1-94-crore-634686',
 'https://www.indiatvnews.com/auto/audi-suv-q8-launch-specifications-price-audi-car-launch-655567',
 'https://www.indiatvnews.com/auto/audi-to-bring-entry-level-suv-q2-to-india-this-festive-season-647815',
 'https://www.indiatvnews.com/auto/bill-gates-electrical-vehicles-zero-carbon-coronavirus-pandemic-644596',
 'https://www.indiatvnews.com/auto/bmw-3-series-gran-turismo-shadow-edition-launched-in-india-at-rs-42-5-lakh-643345',
 'https://www.indiatvnews.com/auto/bmw-cars-price-up-november-all-you-need-to-know-653659',
 'https://www.indiatvnews.com/auto/bmw-s-1000-xr-launched-in-india-priced-at-rs-20-9-lakh-634669',
 'https://www.indiatvnews.com/

In [69]:
class MakeArticles:
    def __init__(self,urls_file_path : str, save_file_path : str):
        self.urls = []
        self.save_file_path = save_file_path
        with open(urls_file_path , "r") as f:
            for line in f:
                self.urls.append(line.split("\n")[0])
        self.articles = []
    def _save_dict(self,name : str , dct : dict):
        save_dict(dct , os.path.join(self.save_file_path, name + ".pkl"))
    
    def _convert_to_dict(self, article : Article):
        return  dict(
            name = article.url,
            title = article.title,
            text = article.text,
            authors = article.authors,
        )
    def build(self):
        failed = 0
        successful = 0
        for index,url in tqdm(enumerate(self.urls)):
            try:
                article = Article(url)
                article.download()
                article.parse()
                article = self._convert_to_dict(article)
                self._save_dict(str(index),article)
                successful += 1
            except:
                failed += 1
                print("{} failed".format(failed))
                continue

        print("num extracted = {} num failed = {}".format(successful, failed))
        

In [70]:
mk_articles = MakeArticles("./ITV_urls.txt", "./ITV_articles")

In [71]:
mk_articles.build()

590it [06:55,  2.57s/it]

1 failed


602it [07:11,  2.77s/it]

2 failed


1805it [20:57,  2.52s/it]

...

16 failed


15719it [2:51:46,  2.59s/it]

17 failed


16154it [2:55:59,  1.53it/s]

num extracted = 16137 num failed = 17


In [12]:
data = load_dict(os.path.join("./NDTV_articles", "0.pkl"))